<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/292_HITL_AgentState.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human-in-the-Loop Collaboration Orchestrator Agent

In [ ]:
# ============================================================================
# Human-in-the-Loop Collaboration Orchestrator Agent
# ============================================================================

class HITLOrchestratorState(TypedDict, total=False):
    """State for Human-in-the-Loop Collaboration Orchestrator Agent"""

    # Input fields
    task_id: Optional[str]                    # Specific task to process (None = process all)

    # Goal & Planning fields (MVP: Fixed goal, template-based plan)
    goal: Dict[str, Any]                      # Goal definition (from goal_node)
    plan: List[Dict[str, Any]]                # Execution plan (from planning_node)

    # Data Ingestion
    tasks: List[Dict[str, Any]]               # All tasks to process
    # Structure per task:
    # {
    #   "task_id": "task_001",
    #   "task_type": "document_classification",
    #   "task_description": "...",
    #   "domain": "customer_support",
    #   "risk_level": "low" | "medium" | "high",
    #   "timestamp": "2025-01-10T09:15:00Z"
    # }

    agent_outputs: List[Dict[str, Any]]       # Agent outputs for tasks
    # Structure per output:
    # {
    #   "task_id": "task_001",
    #   "agent_output": {
    #     "predicted_label": "...",
    #     "explanation": "..."
    #   },
    #   "confidence_score": 0.91
    # }

    routing_policy: Dict[str, Any]            # Routing policy rules
    # Structure:
    # {
    #   "policy_id": "default_routing_policy_v1",
    #   "description": "...",
    #   "rules": [
    #     {
    #       "rule_id": "rule_001",
    #       "priority": 1,
    #       "conditions": {"risk_level": "high"},
    #       "action": "escalate" | "auto_approve" | "human_review",
    #       "assigned_human_role": "senior_manager" | "domain_reviewer" | null
    #     }
    #   ]
    # }

    # Task-Output Mapping (for fast lookup)
    task_output_lookup: Dict[str, Dict[str, Any]]  # task_id -> agent_output

    # Routing Decisions
    routing_decisions: List[Dict[str, Any]]   # Routing decisions for each task
    # Structure per decision:
    # {
    #   "task_id": "task_001",
    #   "risk_level": "low",
    #   "confidence_score": 0.91,
    #   "routing_decision": "auto_approve" | "human_review" | "escalate",
    #   "assigned_human_role": Optional[str],
    #   "rule_applied": "rule_004",
    #   "reasoning": "Low risk, high confidence"
    # }

    # Human Review Processing
    pending_reviews: List[Dict[str, Any]]     # Tasks awaiting human review
    # Structure per review:
    # {
    #   "task_id": "task_002",
    #   "agent_output": {...},
    #   "confidence_score": 0.68,
    #   "risk_level": "medium",
    #   "assigned_human_role": "domain_reviewer",
    #   "requested_at": "2025-01-10T09:30:00Z"
    # }

    human_reviews: List[Dict[str, Any]]       # Human review decisions
    # Structure per review:
    # {
    #   "review_id": "review_001",
    #   "task_id": "task_002",
    #   "human_role": "domain_reviewer",
    #   "human_decision": "approve" | "override" | "modify" | "reject",
    #   "human_feedback": "...",
    #   "confidence_assessment": "high" | "medium" | "low",
    #   "timestamp": "2025-01-10T09:42:00Z"
    # }

    # Final Decisions
    final_decisions: List[Dict[str, Any]]     # Final decisions for all tasks
    # Structure per decision:
    # {
    #   "task_id": "task_001",
    #   "final_decision": "approved" | "rejected" | "modified" | "override_approved",
    #   "decision_source": "agent" | "human",
    #   "confidence_score": 0.91,
    #   "risk_level": "low",
    #   "human_involved": false,
    #   "latency_seconds": 12
    # }

    # Audit Logging
    audit_logs: List[Dict[str, Any]]          # Complete audit trail
    # Structure per log:
    # {
    #   "log_id": "log_001",
    #   "task_id": "task_001",
    #   "risk_level": "low",
    #   "confidence_score": 0.91,
    #   "routing_decision": "auto_approve",
    #   "human_involved": false,
    #   "final_decision": "approved",
    #   "decision_source": "agent",
    #   "latency_seconds": 12,
    #   "timestamp": "2025-01-10T09:15:12Z"
    # }

    # Summary Metrics
    summary_metrics: Dict[str, Any]           # Summary statistics
    # Structure:
    # {
    #   "total_tasks": 5,
    #   "auto_approved_count": 2,
    #   "human_reviewed_count": 3,
    #   "escalated_count": 1,
    #   "average_confidence_score": 0.82,
    #   "average_latency_seconds": 450,
    #   "human_override_count": 1
    # }

    # Output
    orchestrator_report: str                  # Final markdown report
    report_file_path: Optional[str]            # Path to saved report file

    # Metadata
    errors: List[str]                         # Any errors encountered
    processing_time: Optional[float]           # Time taken to process


@dataclass
class HITLOrchestratorConfig:
    """Configuration for Human-in-the-Loop Collaboration Orchestrator Agent"""
    llm_model: str = os.getenv("LLM_MODEL", "gpt-4o-mini")
    temperature: float = 0.3
    reports_dir: str = "output/hitl_orchestrator_reports"  # Where to save reports

    # Data file paths
    data_dir: str = "data"
    tasks_file: str = "tasks.json"
    agent_outputs_file: str = "agent_outputs.json"
    routing_policy_file: str = "routing_policy.json"
    human_reviews_file: str = "human_reviews.json"  # Optional: existing reviews
    audit_logs_file: str = "audit_logs.json"  # Optional: existing logs

    # Routing Settings
    default_routing_policy: str = "default_routing_policy_v1"  # Policy to use

    # Human Review Settings
    auto_approve_for_testing: bool = True  # Auto-approve for testing (MVP)
    human_review_timeout_minutes: int = 60  # Max time to wait for review

    # Audit Settings
    enable_audit_logging: bool = True  # Enable audit trail
    audit_log_retention_days: int = 90  # Days to retain audit logs




Think of this code as **designing the clipboard and rulebook** for a very organized manager who coordinates AI and humans.

---

# 🧠 Big Picture First (Very Important)

This code does **NOT**:

* make decisions
* run AI models
* talk to humans

This code **defines the structure of information** your orchestrator will carry around while it works.

👉 It’s like designing:

* the **forms** people fill out
* the **folders** documents go into
* the **labels** on filing cabinets

Before you build a factory, you design how materials flow through it.

---

# Part 1: `HITLOrchestratorState`

## 🗂️ “The Master Clipboard”

```python
class HITLOrchestratorState(TypedDict, total=False):
```

### What this is (plain English)

This is a **giant shared notebook** that your agent uses while it runs.

Everything the orchestrator:

* receives
* decides
* sends to humans
* logs for auditing

…gets written into this one “state”.

If your orchestrator were a human project manager, this would be **their clipboard**.

---

## 🧩 Why use a “State” at all?

Because agents don’t work in one step.

They go through phases:

1. Load data
2. Make decisions
3. Ask humans
4. Wait
5. Finalize
6. Report

The **state** is how the agent remembers what has already happened.

Without state:

* the agent forgets
* can’t explain decisions
* can’t audit itself
* can’t improve

---

## 🧠 `TypedDict` (ignore Python details)

This just means:

> “This notebook has labeled sections, and each section expects a certain type of information.”

Like:

* “This page is for phone numbers”
* “This page is for dates”
* “This page is for notes”

It prevents chaos.

---

# 🔹 Input Fields

```python
task_id: Optional[str]
```

### Meaning

* If `None` → process **all tasks**
* If set → process **one specific task**

📌 This lets you:

* run batch jobs
* or debug a single task

---

# 🔹 Goal & Planning Fields

```python
goal: Dict[str, Any]
plan: List[Dict[str, Any]]
```

### Conceptually

Even though this is an MVP, you’re **thinking like a real agent**.

* `goal` = *What am I trying to achieve?*
* `plan` = *What steps will I take?*

Right now:

* The goal is fixed
* The plan is template-based

Later:

* The agent could generate its own plans

This is **agent thinking scaffolding**.

---

# 🔹 Data Ingestion

```python
tasks
agent_outputs
routing_policy
```

### Plain English

This is the **raw material** your orchestrator works with.

Think of it like:

* Tasks = homework assignments
* Agent outputs = student answers
* Routing policy = teacher rules

The orchestrator **never guesses**.
It only works with **explicit data**.

This is how you build trust.

---

# 🔹 `task_output_lookup`

```python
task_output_lookup: Dict[str, Dict[str, Any]]
```

### Why this exists

This is a **speed trick**.

Instead of:

* searching through a long list every time

You say:

> “Give me the agent output for task_003 immediately.”

Conceptually:

* It’s an index
* Like a table of contents

You’re teaching the agent to be **efficient**, not clever.

---

# 🔹 Routing Decisions

```python
routing_decisions
```

### This is HUGE conceptually

This is where your agent answers:

> “What did I decide to do with this task — and why?”

Each record contains:

* the decision
* the rule used
* the reasoning

📌 This is what makes your agent:

* explainable
* debuggable
* enterprise-safe

Without this:

* decisions feel magical
* humans don’t trust the system

---

# 🔹 Human Review Processing

```python
pending_reviews
human_reviews
```

### Think of this like a waiting room

* `pending_reviews` = tasks waiting for a human
* `human_reviews` = human responses that came back

This separation is **very important**.

It lets your system:

* pause safely
* resume later
* handle async humans

This is how real companies work.

---

# 🔹 Final Decisions

```python
final_decisions
```

### Why this is separate from routing

Routing says:

> “Who should decide?”

Final decisions say:

> “What actually happened?”

This allows:

* overrides
* corrections
* accountability

This is the **moment of truth**.

---

# 🔹 Audit Logs

```python
audit_logs
```

### Think like a lawyer or regulator

If someone asks:

* “Why was this approved?”
* “Who approved it?”
* “How long did it take?”

This is where the answer lives.

This is **not optional** in real systems.

---

# 🔹 Summary Metrics

```python
summary_metrics
```

### This is for humans, not machines

It answers:

* How much work did we automate?
* Where are humans still needed?
* Is confidence improving?

This is what executives look at.

---

# 🔹 Output

```python
orchestrator_report
report_file_path
```

This is the **story** of what happened:

* readable
* shareable
* reviewable

Agents that can’t explain themselves don’t survive in enterprises.

---

# 🔹 Metadata

```python
errors
processing_time
```

Because real systems fail.

And **honest systems admit it**.

---

# Part 2: `HITLOrchestratorConfig`

## ⚙️ “The Control Panel”

```python
@dataclass
class HITLOrchestratorConfig:
```

### What this is

This is **how you tune the agent without changing its brain**.

Think:

* settings menu
* preferences
* environment switches

---

## 🔹 Model Settings

```python
llm_model
temperature
```

Even though your orchestrator isn’t “thinking” yet:

* you’re preparing it to explain
* summarize
* generate reports

This is future-proofing.

---

## 🔹 File Paths

```python
tasks_file
agent_outputs_file
...
```

This makes your system:

* reproducible
* testable
* portable

Change data → behavior changes
Code stays the same

That’s good architecture.

---

## 🔹 Routing & Human Review Settings

```python
auto_approve_for_testing
human_review_timeout_minutes
```

### This is *very smart* for an MVP

It lets you:

* simulate humans
* test end-to-end flows
* avoid blocking during development

Real systems always have **test modes**.

---

## 🔹 Audit Settings

```python
enable_audit_logging
audit_log_retention_days
```

This is **enterprise thinking**.

You’re planning for:

* storage limits
* compliance rules
* long-term operations

Even if you don’t use it yet — designing for it matters.

---

# 🧠 The Big Takeaway (Most Important)

This code is **not about Python**.

It’s about:

* designing trust
* making decisions traceable
* coordinating humans and machines safely

You are building:

> a *collaboration system*, not an AI toy.


